In [5]:
print("Hello, Jupyter Notebook!")

Hello, Jupyter Notebook!


In [7]:
!pip install opencv-python
!pip install easyocr
import os
import cv2
import numpy as np
from PIL import Image
import pytesseract
import easyocr
from datetime import datetime

def preprocess_image(image):
    # Convert PIL Image to cv2 format
    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply CLAHE to improve the contrast of the image
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)
    
    # Apply adaptive thresholding
    gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    
    # Apply morphological operations to remove noise and improve text connectivity
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    gray = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
    
    # Apply a median blur to remove small noise
    gray = cv2.medianBlur(gray, 3)
    
    return Image.fromarray(gray)

def improved_ocr(image, lang='en', use_easyocr=False):
    if use_easyocr:
        reader = easyocr.Reader([lang], gpu=False)
        try:
            # Perform OCR using EasyOCR
            result = reader.readtext(np.array(image), detail=0)
            full_text = ' '.join(result)
            return {
                'text': full_text,
                'confidence': None,  # EasyOCR does not provide a confidence score per word
                'raw_data': result
            }
        except Exception as e:
            print(f"EasyOCR Error: {str(e)}")
            return None
    else:
        # Configure pytesseract
        custom_config = r'--oem 3 --psm 6'
        
        # Preprocess image
        processed_img = preprocess_image(image)
        
        try:
            # Get OCR text with confidence
            data = pytesseract.image_to_data(processed_img, lang=lang, config=custom_config, output_type=pytesseract.Output.DICT)
            
            # Extract text with confidence scores
            full_text = []
            confidences = []
            
            for i, word_conf in enumerate(data['conf']):
                if float(word_conf) > 0:  # Filter out low confidence results
                    word = data['text'][i]
                    if word.strip():  # Only add non-empty words
                        full_text.append(word)
                        confidences.append(float(word_conf))
            
            result = {
                'text': ' '.join(full_text),
                'confidence': sum(confidences) / len(confidences) if confidences else 0,
                'raw_data': data
            }
            return result
            
        except Exception as e:
            print(f"OCR Error: {str(e)}")
            return None

def save_ocr_results(result, folder='ocr_results'):
    if not os.path.exists(folder):
        os.makedirs(folder)
        
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'ocr_result_{timestamp}.txt'
    filepath = os.path.join(folder, filename)
    
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(f"OCR Text:\n{result['text']}\n\n")
            if result['confidence'] is not None:
                f.write(f"Average Confidence: {result['confidence']:.2f}%\n")
        print(f"OCR results saved to: {filepath}")
    except Exception as e:
        print(f"Error saving results: {str(e)}")

# Process image and save results
image_path = 'keywords.png'
image = Image.open(image_path)
ocr_result = improved_ocr(image, use_easyocr=True)

if ocr_result:
    save_ocr_results(ocr_result)

Using CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% CompleteOCR results saved to: ocr_results/ocr_result_20241115_114728.txt
